In [1]:
%%bash
# Create Service account that will be used for Vault to check against K8S API
cat > vault-auth-service-account.yaml <<EOF
apiVersion: v1
kind: ServiceAccount
metadata:
  name: vault-auth
  namespace: default
---
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRoleBinding
metadata:
  name: role-tokenreview-binding
  namespace: default
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: system:auth-delegator
subjects:
- kind: ServiceAccount
  name: vault-auth
  namespace: default
EOF

kubectl apply --filename vault-auth-service-account.yaml

serviceaccount/vault-auth created
clusterrolebinding.rbac.authorization.k8s.io/role-tokenreview-binding created


In [2]:
%%bash
# Define service account secret
cat > vault-auth-secret.yaml <<EOF
apiVersion: v1
kind: Secret
metadata:
  name: vault-auth-secret
  annotations:
    kubernetes.io/service-account.name: vault-auth
type: kubernetes.io/service-account-token
EOF

kubectl apply --filename vault-auth-secret.yaml

secret/vault-auth-secret created


In [4]:
%env WORKDIR=/tmp/vault

env: WORKDIR=/tmp/vault


In [5]:
%%bash
cat ${WORKDIR}/cluster-keys.json | jq -r ".root_token"

hvs.oS753uxfMVaEVRjHDqqgaJCh


In [19]:
%env VAULT_ADDR=https://127.0.0.1:8200
%env VAULT_TOKEN=hvs.oS753uxfMVaEVRjHDqqgaJCh
%env VAULT_CACERT=/tmp/vault/vault.ca

env: VAULT_ADDR=https://127.0.0.1:8200
env: VAULT_TOKEN=hvs.oS753uxfMVaEVRjHDqqgaJCh
env: VAULT_CACERT=/tmp/vault/vault.ca


In [29]:
%%bash
## Create K8S Auth method with mock data
vault policy write myapp-kv-ro - <<EOF
path "secret/data/myapp/*" {
    capabilities = ["read", "list"]
}
EOF


Success! Uploaded policy: myapp-kv-ro


In [ ]:
%%bash
vault secrets enable -path=secret kv-v2

In [22]:
%%bash
vault kv put secret/myapp/config \
      username='appuser' \
      password='suP3rsec(et!' \
      ttl='30s'


Success! Enabled the kv-v2 secrets engine at: secret/
====== Secret Path ======
secret/data/myapp/config

======= Metadata =======
Key                Value
---                -----
created_time       2023-04-19T13:55:31.819358046Z
custom_metadata    <nil>
deletion_time      n/a
destroyed          false
version            1


In [27]:
%%bash
# Test Auth method

cat > devwebapp.yaml <<EOF
apiVersion: v1
kind: Pod
metadata:
  name: devwebapp
  labels:
    app: devwebapp
spec:
  serviceAccountName: vault-auth
  containers:
    - name: devwebapp
      image: nginx
      env:
        - name: VAULT_ADDR
          value: "https://vault.vault.svc.cluster.local:8200"
EOF

kubectl apply --filename devwebapp.yaml --namespace default

pod/devwebapp created


In [76]:
%%bash
cat > configmap.yaml <<EOF
apiVersion: v1
data:
  vault-agent-config.hcl: |
    # Comment this out if running as sidecar instead of initContainer
    exit_after_auth = true

    pid_file = "/home/vault/pidfile"

    vault {
        address = "https://vault.vault.svc.cluster.local:8200"
        tls_skip_verify = "True"
    }

    auto_auth {
        # https://developer.hashicorp.com/vault/docs/agent/autoauth/methods
        method "kubernetes" {
            mount_path = "auth/kubernetes"
            config = {
                role = "example"
            }
        }

        sink "file" {
            config = {
                path = "/home/vault/.vault-token"
            }
        }
    }

    template {
    destination = "/etc/secrets/index.html"
    contents = <<EOT
    <html>
    <body>
    <p>Some secrets:</p>
    {{- with secret "secret/data/myapp/config" }}
    <ul>
    <li><pre>username: {{ .Data.data.username }}</pre></li>
    <li><pre>password: {{ .Data.data.password }}</pre></li>
    </ul>
    {{ end }}
    </body>
    </html>
    EOT
    }
kind: ConfigMap
metadata:
  name: example-vault-agent-config
  namespace: default
EOF

In [77]:
%%bash
kubectl apply --filename configmap.yaml
kubectl get configmap example-vault-agent-config --output yaml

configmap/example-vault-agent-config unchanged
apiVersion: v1
data:
  vault-agent-config.hcl: |
    # Comment this out if running as sidecar instead of initContainer
    exit_after_auth = true

    pid_file = "/home/vault/pidfile"

    vault {
        address = "https://vault.vault.svc.cluster.local:8200"
        tls_skip_verify = "True"
    }

    auto_auth {
        method "kubernetes" {
            mount_path = "auth/kubernetes"
            config = {
                role = "example"
            }
        }

        sink "file" {
            config = {
                path = "/home/vault/.vault-token"
            }
        }
    }

    template {
    destination = "/etc/secrets/index.html"
    contents = <<EOT
    <html>
    <body>
    <p>Some secrets:</p>
    {{- with secret "secret/data/myapp/config" }}
    <ul>
    <li><pre>username: {{ .Data.data.username }}</pre></li>
    <li><pre>password: {{ .Data.data.password }}</pre></li>
    </ul>
    {{ end }}
    </body>
    </html>
   

In [80]:
%%bash
cat > example-k8s-spec.yaml <<EOF
apiVersion: v1
kind: Pod
metadata:
  name: vault-agent-example
  namespace: default
spec:
  serviceAccountName: vault-auth

  volumes:
    - configMap:
        items:
          - key: vault-agent-config.hcl
            path: vault-agent-config.hcl
        name: example-vault-agent-config
      name: config
    - emptyDir: {}
      name: shared-data

  initContainers:
    - args:
        - agent
        - -config=/etc/vault/vault-agent-config.hcl
        - -log-level=debug
      image: docker.io/hashicorp/vault
      name: vault-agent
      volumeMounts:
        - mountPath: /etc/vault
          name: config
        - mountPath: /etc/secrets
          name: shared-data

  containers:
    - image: nginx
      name: nginx-container
      ports:
        - containerPort: 80
      volumeMounts:
        - mountPath: /usr/share/nginx/html
          name: shared-data
EOF


In [81]:
%%bash
kubectl apply --filename example-k8s-spec.yaml

pod/vault-agent-example created


In [ ]:
%%bash
#kubectl -n $VAULT_K8S_NAMESPACE get service vault
kubectl port-forward pod/vault-agent-example 8080:80

In [83]:
%%bash
kubectl delete --filename example-k8s-spec.yaml
kubectl delete -f devwebapp.yaml

Error from server (NotFound): error when deleting "example-k8s-spec.yaml": pods "vault-agent-example" not found


pod "devwebapp" deleted
